In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential
from keras.regularizers import l1

Using TensorFlow backend.


In [2]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

In [3]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [4]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [5]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 50 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 100

In [6]:
model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
#         dropout=0.05,
#         activity_regularizer=l1(0.001),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam") #rmsprop

In [7]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/3
158773/158773 [==============================] - 6s 37us/step - loss: 2.4651
Epoch 2/3
158773/158773 [==============================] - 5s 35us/step - loss: 2.0419
Epoch 3/3
158773/158773 [==============================] - 5s 30us/step - loss: 1.8728
Генерация из посева: time as we
time as west and the master and the master and the master and the master and the master and the master and the==================================================
Итерация #: 1
Epoch 1/3
158773/158773 [==============================] - 5s 30us/step - loss: 1.7569
Epoch 2/3
158773/158773 [==============================] - 5s 30us/step - loss: 1.6675
Epoch 3/3
158773/158773 [==============================] - 5s 30us/step - loss: 1.5968
Генерация из посева: our advent
our advented the mock turtle so she was a little some the mock turtle so she was a little some the mock turtle==================================================
Итерация #: 2
Epoch 1/3
158773/158773 [=========================

158773/158773 [==============================] - 5s 29us/step - loss: 0.8290
Epoch 3/3
158773/158773 [==============================] - 5s 29us/step - loss: 0.8246
Генерация из посева: d she had 
d she had never been any put of her head that dormouse, who was persecting on the work of the state of choking==================================================
Итерация #: 18
Epoch 1/3
158773/158773 [==============================] - 6s 35us/step - loss: 0.8181
Epoch 2/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.8123
Epoch 3/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.8072
Генерация из посева: and the mo
and the moral of that is--oo she would get to do anything you seem to tell its arm a sort of meaning in it, sa==================================================
Итерация #: 19
Epoch 1/3
158773/158773 [==============================] - 5s 34us/step - loss: 0.8023
Epoch 2/3
158773/158773 [==============================] - 5s 30us/step 

alice had not attending to herself, and shouted out, your said to the hatter, and he shall smalp bewankly. --i==================================================
Итерация #: 35
Epoch 1/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.6487
Epoch 2/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.6472
Epoch 3/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.6443
Генерация из посева: , said the
, said the mock turtle in the distance. copied myshous fold the other side of the copyright roysing the cook. ==================================================
Итерация #: 36
Epoch 1/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.6436
Epoch 2/3
158773/158773 [==============================] - 5s 29us/step - loss: 0.6429
Epoch 3/3
158773/158773 [==============================] - 5s 30us/step - loss: 0.6391
Генерация из посева: aid the ca
aid the caterpillar took the whoter is all him anyw what theyr